In [1]:
# Required Libraries
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from math import sqrt
import random

# Prime Generator Class
class PrimeGenerator:
    def __init__(self):
        self.small_primes = self.sieve_of_eratosthenes(1000)
        self.model = self.build_model()

    def build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def sieve_of_eratosthenes(self, n):
        sieve = [True] * (n+1)
        sieve[0] = sieve[1] = False
        for i in range(2, int(sqrt(n)) + 1):
            if sieve[i]:
                sieve[i*i :: i] = [False] * len(sieve[i*i :: i])
        return [i for i, is_prime in enumerate(sieve) if is_prime]

    def is_prime(self, n, k=5):
        if n < 2:
            return False
        for p in self.small_primes:
            if n % p == 0:
                return n == p
        d = n - 1
        s = 0
        while d % 2 == 0:
            d //= 2
            s += 1
        for _ in range(k):
            a = random.randint(2, n - 1)
            x = pow(a, d, n)
            if x == 1 or x == n - 1:
                continue
            for __ in range(s - 1):
                x = pow(x, 2, n)
                if x == n - 1:
                    break
            else:
                return False
        return True

    def generate_features(self, n):
        features = [
            n % 2,
            n % 3,
            n % 5,
            n % 7,
            int(str(n)[-1]),  # last digit
            len(str(n)),      # number of digits
            sum(int(d) for d in str(n)),  # digit sum
            n % 10,
            int(str(n)[0]),  # first digit
            sum(1 for p in self.small_primes if n % p == 0)
        ]
        return np.array(features).reshape(1, -1)

    def train_model(self, X, y):
        self.model.fit(X, y, epochs=10, batch_size=32)

    def generate_primes(self, start, end, batch_size=1000):
        primes = []
        if end <= 1e6:
            sieve = [True] * (end - start + 1)
            for p in self.small_primes:
                if p * p > end:
                    break
                first_multiple = max(p*p, ((start + p - 1) // p) * p)
                sieve[first_multiple - start :: p] = [False] * len(sieve[first_multiple - start :: p])
            primes.extend([i + start for i, is_prime in enumerate(sieve) if is_prime])
            return primes

        candidates = range(start, end + 1)
        candidates = [n for n in candidates if n % 2 != 0 and n % 3 != 0 and n % 5 != 0]

        features = np.array([self.generate_features(n)[0] for n in candidates])
        predictions = self.model.predict(features, verbose=0)
        likely_primes = [n for n, pred in zip(candidates, predictions) if pred > 0.5]

        primes = [n for n in likely_primes if self.is_prime(n)]

        return primes

# Utility Function to Generate Training Data
def generate_training_data(generator, num_samples=10000):
    X, y = [], []
    for _ in range(num_samples):
        n = random.randint(10, 1000000)
        features = generator.generate_features(n)[0]
        label = int(generator.is_prime(n))  # 1 if prime, else 0
        X.append(features)
        y.append(label)
    return np.array(X), np.array(y)

# Main Execution
if __name__ == "__main__":
    generator = PrimeGenerator()

    # Step 1: Generate training data and train the model
    print("Generating training data...")
    X_train, y_train = generate_training_data(generator)
    print("Training model...")
    generator.train_model(X_train, y_train)

    # Step 2: Generate primes using the trained model
    print("Generating primes between 1,000,000 and 1,001,000...")
    primes = generator.generate_primes(1000000, 1001000)
    print("Primes found:")
    print(primes)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Generating training data...
Training model...
Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9324 - loss: 0.2346
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0214
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9997 - loss: 0.0038
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9998 - loss: 0.0020
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9999 - loss: 0.0013
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9998 - loss: 0.0014
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9999 - loss: 9.6133e-04
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9998 - loss: 0.0017
Epoch 9/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.2197e-04
Epoch 10/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 2.7917e-04
Generating primes between 1,000,000 and 1,001,000...
Primes found:
[1000003, 1000

In [2]:
primes = generator.generate_primes(1000, 2000)
print("Generated primes:")
for i, prime in enumerate(primes, start=1):
    print(f"{prime:5d}", end="  ")
    if i % 10 == 0:
        print()

Generated primes:
 1009   1013   1019   1021   1031   1033   1039   1049   1051   1061  
 1063   1069   1087   1091   1093   1097   1103   1109   1117   1123  
 1129   1151   1153   1163   1171   1181   1187   1193   1201   1213  
 1217   1223   1229   1231   1237   1249   1259   1277   1279   1283  
 1289   1291   1297   1301   1303   1307   1319   1321   1327   1361  
 1367   1373   1381   1399   1409   1423   1427   1429   1433   1439  
 1447   1451   1453   1459   1471   1481   1483   1487   1489   1493  
 1499   1511   1523   1531   1543   1549   1553   1559   1567   1571  
 1579   1583   1597   1601   1607   1609   1613   1619   1621   1627  
 1637   1657   1663   1667   1669   1693   1697   1699   1709   1721  
 1723   1733   1741   1747   1753   1759   1777   1783   1787   1789  
 1801   1811   1823   1831   1847   1861   1867   1871   1873   1877  
 1879   1889   1901   1907   1913   1931   1933   1949   1951   1973  
 1979   1987   1993   1997   1999  